In [1]:
!pip install findspark

In [2]:
import os
import findspark

In [3]:
findspark.init()

In [4]:
from pyspark.sql import SparkSession
# Initialize SparkSession

spark = SparkSession.builder \
    .appName("Hello1234") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

In [19]:
spark.stop()

In [9]:
# Subscribe to 1 topic
kafka_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka-broker-0:9092,kafka-broker-1:9092") \
  .option("subscribe", "test") \
  .option("startingOffsets", "earliest") \
  .load()


In [16]:
from pyspark.sql.functions import col, from_json

In [17]:
# Cast key and value columns to string
kafka_df = kafka_df \
    .withColumn("key", col("key").cast("string")) \
    .withColumn("value", col("value").cast("string"))


In [31]:
query = kafka_df \
    .writeStream \
    .outputMode("update") \
    .format("memory") \
    .queryName("mem_results") \
    .start()

In [35]:
spark.sql('show tables').show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|         |mem_results|      false|
+---------+-----------+-----------+



In [36]:
spark.sql('select * from mem_results').show()

+------+------+-----+---------+------+--------------------+-------------+
|   key| value|topic|partition|offset|           timestamp|timestampType|
+------+------+-----+---------+------+--------------------+-------------+
|     a|     b| test|        0|     0|2024-07-15 04:52:...|            0|
|   abc|    dc| test|        0|     1|2024-07-15 04:54:...|            0|
|qweqwe|qweqwe| test|        0|     2|2024-07-15 04:58:...|            0|
+------+------+-----+---------+------+--------------------+-------------+

